#  Noisy data
In this notebook, we will explore how data quality can affect predictions. We will simulate examples where noise is introduced in the test set, the training set, or both, and examine
the effects on model performance.   

In [2]:
import numpy as np 
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import modular.samples_setup as cs
from modular import engine
from modular import model_builder
from modular import extra_functions as ef

#### 1. Data simulation

Let's start by simulating some data. We will simulate a data set containing 5000 samples each of
squares and circles. We will save 20% of the data for test and we will use the rest for
train the models

In [3]:
### Simulate data 
SEED = 262
n_samples = [5000]*2

output = cs.generate_sample(n=n_samples, noise_prop=0, var=0, seed=SEED)
images, labels = (output['images'], output['labels'])

# Split test and train
n_test = int(sum(n_samples)*0.2)
test_index = np.arange(n_test)
train_index = np.arange(n_test, sum(n_samples))

images_test = images[test_index]
images_train = images[train_index]

label_test = labels[test_index]
label_train = labels[train_index]

#### 2. Scenario with no noise
Let's start by training the model in a scenario where both the training and the test sets are noise-free, ensuring all samples are perfectly clear


In [4]:
# To reduce variability when re-running 
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

BATCH_SIZE = 50
EPOCHS = 12
loss_fn = nn.CrossEntropyLoss()

# Create tensor
X_test = torch.from_numpy(images_test).type(torch.float)
X_train = torch.from_numpy(images_train).type(torch.float)

y_train = torch.from_numpy(label_train).type(torch.long)
y_test = torch.from_numpy(label_test).type(torch.long)

## Add channel at dimension 1 (greyscale)
X_train = X_train.unsqueeze(1)  
X_test = X_test.unsqueeze(1)  
        
train_dataset = torch.utils.data.TensorDataset(X_train,y_train)
test_dataset = torch.utils.data.TensorDataset(X_test,y_test)
        
# Create data loader and turn datasets into iterables (batches)
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
    ) 
                           
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
    )

# Initialize model and optimizer
model_1 =  model_builder.TVGG(
    input_shape=1,  
    hidden_units=10, 
    output_shape=2
    )

optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.1)

output_1 = engine.train_test_loop(
    model=model_1,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer, 
    loss_fn=loss_fn,
    epochs=EPOCHS,
    print_b=True
    )

Epoch: 1 | test_ce: 5.07312 | test_acc: 75.6000
Epoch: 2 | test_ce: 4.91997 | test_acc: 75.3500
Epoch: 3 | test_ce: 4.66924 | test_acc: 78.8500
Epoch: 4 | test_ce: 3.19666 | test_acc: 83.8000
Epoch: 5 | test_ce: 0.96844 | test_acc: 95.2500
Epoch: 6 | test_ce: 0.52768 | test_acc: 96.8500
Epoch: 7 | test_ce: 0.48824 | test_acc: 97.4000
Epoch: 8 | test_ce: 0.47484 | test_acc: 97.4000
Epoch: 9 | test_ce: 0.45586 | test_acc: 97.4000
Epoch: 10 | test_ce: 0.46344 | test_acc: 97.4000
Epoch: 11 | test_ce: 0.44879 | test_acc: 97.3500
Epoch: 12 | test_ce: 0.45594 | test_acc: 97.4000


Quite good!

#### 3. Scenario with noise in the test set
What happens if out test set is entirely noisy, while the training set remains noise-free?

In [5]:
# Adding noise only to the test set
VAR = 0.15
BATCH_SIZE = 50
EPOCHS = 12
loss_fn = nn.CrossEntropyLoss()

# Create copy of image sets

images_test_e1 = images_test.copy()
images_train_e1 = images_train.copy()

label_test = labels[test_index]
label_train = labels[train_index]

# Add noise        
for i in range(len(images_test)):
        images_test_e1[i] = ef.add_gaussian_noise(
                image=images_test_e1[i],
                var=VAR
        )

# Create tensor
X_test = torch.from_numpy(images_test_e1).type(torch.float)
X_train = torch.from_numpy(images_train_e1).type(torch.float)

y_test = torch.from_numpy(label_test).type(torch.long)
y_train = torch.from_numpy(label_train).type(torch.long)

## Add channel at dimension 1 (greyscale)
X_train = X_train.unsqueeze(1)  
X_test = X_test.unsqueeze(1)  
        
train_dataset = torch.utils.data.TensorDataset(X_train,y_train)
test_dataset = torch.utils.data.TensorDataset(X_test,y_test)
        
# Create data loader and turn datasets into iterables (batches)
train_dataloader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False
        ) 
                            
test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False
        )

# Initialize model and optimizer
model_2 =  model_builder.TVGG(
        input_shape=1,  
        hidden_units=10, 
        output_shape=2
        )

optimizer = torch.optim.SGD(params=model_2.parameters(), lr=0.1)

output_2 = engine.train_test_loop(
        model=model_2,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader, 
        optimizer=optimizer,
        loss_fn=loss_fn,
        epochs=EPOCHS,
        print_b=True
        )

Epoch: 1 | test_ce: 6.02080 | test_acc: 67.5500
Epoch: 2 | test_ce: 5.67422 | test_acc: 68.1000
Epoch: 3 | test_ce: 5.91881 | test_acc: 66.0500
Epoch: 4 | test_ce: 7.33350 | test_acc: 58.1000
Epoch: 5 | test_ce: 8.68649 | test_acc: 50.2000
Epoch: 6 | test_ce: 8.29231 | test_acc: 50.8000
Epoch: 7 | test_ce: 7.98258 | test_acc: 51.5000
Epoch: 8 | test_ce: 7.58947 | test_acc: 53.8500
Epoch: 9 | test_ce: 7.94438 | test_acc: 51.6000
Epoch: 10 | test_ce: 7.96077 | test_acc: 50.9500
Epoch: 11 | test_ce: 7.94166 | test_acc: 51.3000
Epoch: 12 | test_ce: 7.90330 | test_acc: 51.4500


It really worsens!

#### 3. Scenario with noise in the train set
What happens if our training set is entirely noisy while the test set remains noise-free?


In [6]:
# Adding noise only to the train set
VAR = 0.15
BATCH_SIZE = 50
EPOCHS = 12
loss_fn = nn.CrossEntropyLoss()

# Create copy of image sets
images_test_e2 = images_test.copy()
images_train_e2 = images_train.copy()

label_test = labels[test_index]
label_train = labels[train_index]

# add Gaussian Noise       
for i in range(len(images_test)):
        images_train_e2[i] = ef.add_gaussian_noise(
                image=images_train_e2[i],
                var=VAR
                )

# Create tensor
X_test = torch.from_numpy(images_test_e2).type(torch.float)
X_train = torch.from_numpy(images_train_e2).type(torch.float)

y_test = torch.from_numpy(label_test).type(torch.long)
y_train = torch.from_numpy(label_train).type(torch.long)

## Add channel at dimension 1 (greyscale)
X_train = X_train.unsqueeze(1)  
X_test = X_test.unsqueeze(1)  
        
train_dataset = torch.utils.data.TensorDataset(X_train,y_train)
test_dataset = torch.utils.data.TensorDataset(X_test,y_test)
        
# Create data loader and turn datasets into iterables (batches)
train_dataloader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False
        ) 
                            
test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False
        )

# Initialize model and optimizer
model_3 =  model_builder.TVGG(
        input_shape=1,  
        hidden_units=10, 
        output_shape=2
        )

optimizer = torch.optim.SGD(params=model_3.parameters(), lr=0.1)

output_3 = engine.train_test_loop(
        model=model_3,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader, 
        optimizer=optimizer, 
        loss_fn=loss_fn,
        epochs=EPOCHS,
        print_b=True
        )

Epoch: 1 | test_ce: 6.57604 | test_acc: 75.3000
Epoch: 2 | test_ce: 5.96900 | test_acc: 75.3000
Epoch: 3 | test_ce: 5.64955 | test_acc: 72.9000
Epoch: 4 | test_ce: 5.27190 | test_acc: 73.0500
Epoch: 5 | test_ce: 4.25713 | test_acc: 76.9000
Epoch: 6 | test_ce: 2.47657 | test_acc: 85.2000
Epoch: 7 | test_ce: 0.88503 | test_acc: 93.9000
Epoch: 8 | test_ce: 0.59424 | test_acc: 96.1500
Epoch: 9 | test_ce: 0.45314 | test_acc: 97.1500
Epoch: 10 | test_ce: 0.46082 | test_acc: 97.4000
Epoch: 11 | test_ce: 0.46469 | test_acc: 97.4500
Epoch: 12 | test_ce: 0.43979 | test_acc: 97.4500


This is not so bad!

#### 4. Scenario with noise in both sets
Let's see an example where we have noisy data in both sets. We will add noise to half
of the training set and half of the test set


In [7]:
VAR =0.15
BATCH_SIZE = 50
EPOCHS = 12
loss_fn = nn.CrossEntropyLoss()

# Create copy of image sets
images_test_e3 = images_test.copy()
images_train_e3 = images_train.copy()

# add noise to half of them 
n_test_noisy = int(len(images_test)*0.5)
n_train_noisy = int(len(images_train)*0.5)

# add Gaussian Noise       
for i in range(n_test_noisy):
        images_test_e3[i] = ef.add_gaussian_noise(
                image=images_test_e3[i],
                var=VAR
                )
        
for i in range(n_train_noisy):
        images_train_e3[i] = ef.add_gaussian_noise(
                image=images_train_e3[i],
                var=VAR
                )

# Create tensor
X_test = torch.from_numpy(images_test_e3).type(torch.float)
X_train = torch.from_numpy(images_train_e3).type(torch.float)

y_test = torch.from_numpy(label_test).type(torch.long)
y_train = torch.from_numpy(label_train).type(torch.long)

## Add channel at dimension 1 (greyscale)
X_train = X_train.unsqueeze(1)  
X_test = X_test.unsqueeze(1)  
        
train_dataset = torch.utils.data.TensorDataset(X_train,y_train)
test_dataset = torch.utils.data.TensorDataset(X_test,y_test)
        
# Create data loader and turn datasets into iterables (batches)
train_dataloader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False
        ) 
                        
test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False
        )


# Initialize model and optimizer
model_4 =  model_builder.TVGG(
        input_shape=1,  
        hidden_units=10, 
        output_shape=2
        )

optimizer = torch.optim.SGD(params=model_4.parameters(), lr=0.1)

output_4 = engine.train_test_loop(
        model=model_4,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader, 
        optimizer=optimizer, 
        loss_fn=loss_fn,
        epochs=EPOCHS,
        print_b=True
        )                        

Epoch: 1 | test_ce: 6.03790 | test_acc: 71.2000
Epoch: 2 | test_ce: 5.70188 | test_acc: 71.4500
Epoch: 3 | test_ce: 5.40449 | test_acc: 71.4000
Epoch: 4 | test_ce: 5.23908 | test_acc: 71.8000
Epoch: 5 | test_ce: 5.18184 | test_acc: 71.4000
Epoch: 6 | test_ce: 3.70739 | test_acc: 78.2000
Epoch: 7 | test_ce: 3.05427 | test_acc: 80.7500
Epoch: 8 | test_ce: 2.75043 | test_acc: 81.8000
Epoch: 9 | test_ce: 2.50962 | test_acc: 84.0500
Epoch: 10 | test_ce: 2.38252 | test_acc: 85.1500
Epoch: 11 | test_ce: 2.41362 | test_acc: 85.2000
Epoch: 12 | test_ce: 2.39461 | test_acc: 85.5500
